# Process Data
#### Daniel Fay

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np

In [2]:
# Load dataset
bus = pd.read_csv('data/interim/bus_times.csv', low_memory=False, index_col=0)

### Create Temporal Features

In [3]:
bus.drop(['trip', 'end_time'],axis=1,inplace=True)
bus['begin_time'] = pd.to_datetime(bus['begin_time'])
bus['trip_time'] = pd.to_timedelta(bus['trip_time'])
bus.head()

bus_line  direction  trip_time          begin_time
0      B11        2.0   00:39:54 2016-01-01 09:11:05
1      B11        2.0   00:38:27 2016-01-01 09:44:02
2      B11        1.0   00:53:54 2016-01-01 09:52:02
3      B11        2.0   00:34:42 2016-01-01 10:10:50
4      B11        1.0   00:41:03 2016-01-01 10:23:00

In [4]:
bus['day_of_week'] = bus['begin_time'].dt.weekday_name
bus['month'] = bus['begin_time'].dt.month
bus['hour'] = bus['begin_time'].dt.hour
bus['date'] = bus['begin_time'].dt.date
bus['day'] = bus['begin_time'].dt.day
bus['minute'] = bus['begin_time'].dt.minute
bus['trip_time'] = bus['trip_time'].dt.total_seconds()/60
bus.head()

bus_line  direction  trip_time          begin_time day_of_week  month  hour  \
0      B11        2.0      39.90 2016-01-01 09:11:05      Friday      1     9   
1      B11        2.0      38.45 2016-01-01 09:44:02      Friday      1     9   
2      B11        1.0      53.90 2016-01-01 09:52:02      Friday      1     9   
3      B11        2.0      34.70 2016-01-01 10:10:50      Friday      1    10   
4      B11        1.0      41.05 2016-01-01 10:23:00      Friday      1    10   

         date  day  minute  
0  2016-01-01    1      11  
1  2016-01-01    1      44  
2  2016-01-01    1      52  
3  2016-01-01    1      10  
4  2016-01-01    1      23

In [5]:
traf_int = {}
for month in range(6,10):
    traf_int[month] = 'PeakAM'
    
for month in range(10,16):
    traf_int[month] = 'MidDay'
    
for month in range(16,20):
    traf_int[month] = 'PeakPM'
    
for month in range(20,25):
    traf_int[month] = 'Night'
    
for month in range(6):
    traf_int[month] = 'Night'

In [6]:
bus['time_period'] = bus['hour'].map(traf_int)
bus.sort_values(['bus_line','begin_time'], inplace=True)
bus = bus.loc[bus['trip_time'] < 150]
bus = bus.loc[bus['trip_time'] > 5]
bus.head()

bus_line  direction  trip_time          begin_time day_of_week  month  \
115      B11        1.0  67.633333 2016-01-01 00:00:13      Friday      1   
107      B11        2.0  36.066667 2016-01-01 00:00:14      Friday      1   
112      B11        2.0  31.000000 2016-01-01 00:00:15      Friday      1   
100      B11        1.0   7.200000 2016-01-01 00:00:17      Friday      1   
109      B11        1.0  23.900000 2016-01-01 00:00:18      Friday      1   

     hour        date  day  minute time_period  
115     0  2016-01-01    1       0       Night  
107     0  2016-01-01    1       0       Night  
112     0  2016-01-01    1       0       Night  
100     0  2016-01-01    1       0       Night  
109     0  2016-01-01    1       0       Night

In [7]:
len(bus)

836951

### Aggregate trips to 1 minute interval

In [8]:
bus = bus.groupby(['bus_line','direction','date','month','day','day_of_week','time_period', \
                    'hour','minute']).mean().reset_index()
bus.head()

bus_line  direction        date  month  day day_of_week time_period  hour  \
0      B11        1.0  2016-01-01      1    1      Friday      MidDay    10   
1      B11        1.0  2016-01-01      1    1      Friday      MidDay    10   
2      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   
3      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   
4      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   

   minute  trip_time  
0      23  41.050000  
1      46  54.783333  
2      13  54.466667  
3      35  40.616667  
4      54  51.150000

### Calculate Mean and Standard Deviation for Trip Times in Each Category

In [9]:
bus_temp = bus.drop(['hour', 'date', 'day','minute'], axis=1)
avg_times = bus_temp.groupby(['bus_line', 'direction', 'month', 'day_of_week', 'time_period']).mean().reset_index()
avg_times.columns = ['bus_line', 'direction', 'month','day_of_week', 'time_period', 'avg_trip_time']

In [10]:
std_times = bus_temp.groupby(['bus_line','direction', 'month', 'day_of_week', 'time_period']).std().reset_index()
std_times.columns = ['bus_line', 'direction', 'month', 'day_of_week', 'time_period', 'std_trip_time']

In [11]:
avg_times.head()

bus_line  direction  month day_of_week time_period  avg_trip_time
0      B11        1.0      1      Friday      MidDay      64.289123
1      B11        1.0      1      Friday       Night      62.360454
2      B11        1.0      1      Friday      PeakAM      55.033333
3      B11        1.0      1      Friday      PeakPM      69.339542
4      B11        1.0      1      Monday      MidDay      67.660468

In [12]:
std_times.head()

bus_line  direction  month day_of_week time_period  std_trip_time
0      B11        1.0      1      Friday      MidDay      16.003063
1      B11        1.0      1      Friday       Night      20.746053
2      B11        1.0      1      Friday      PeakAM       7.915746
3      B11        1.0      1      Friday      PeakPM      13.657906
4      B11        1.0      1      Monday      MidDay      17.047391

In [13]:
bus_times = pd.merge(bus, avg_times, how='left', on=['bus_line', 'direction', 'month', 'day_of_week', 'time_period'])
bus_times = pd.merge(bus_times, std_times, how='left', on=['bus_line', 'direction', 'month', 'day_of_week', 'time_period'])

In [14]:
bus_times['delay_time'] = bus_times['trip_time'] - bus_times['avg_trip_time']

In [15]:
bus_times.head()

bus_line  direction        date  month  day day_of_week time_period  hour  \
0      B11        1.0  2016-01-01      1    1      Friday      MidDay    10   
1      B11        1.0  2016-01-01      1    1      Friday      MidDay    10   
2      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   
3      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   
4      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   

   minute  trip_time  avg_trip_time  std_trip_time  delay_time  
0      23  41.050000      64.289123      16.003063  -23.239123  
1      46  54.783333      64.289123      16.003063   -9.505790  
2      13  54.466667      64.289123      16.003063   -9.822457  
3      35  40.616667      64.289123      16.003063  -23.672457  
4      54  51.150000      64.289123      16.003063  -13.139123

### Calculate Delay

In [16]:
delay = []

for idx, row in bus_times.iterrows():
    if (row['trip_time'] - row['avg_trip_time']) > (2 * row['std_trip_time']):
        delay.append(1)
    elif (row['trip_time'] - row['avg_trip_time']) > (3 * row['std_trip_time']):
        delay.append(2)
    else:
        delay.append(0)

In [17]:
bus_times['delay'] = delay
bus_times.head()

bus_line  direction        date  month  day day_of_week time_period  hour  \
0      B11        1.0  2016-01-01      1    1      Friday      MidDay    10   
1      B11        1.0  2016-01-01      1    1      Friday      MidDay    10   
2      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   
3      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   
4      B11        1.0  2016-01-01      1    1      Friday      MidDay    11   

   minute  trip_time  avg_trip_time  std_trip_time  delay_time  delay  
0      23  41.050000      64.289123      16.003063  -23.239123      0  
1      46  54.783333      64.289123      16.003063   -9.505790      0  
2      13  54.466667      64.289123      16.003063   -9.822457      0  
3      35  40.616667      64.289123      16.003063  -23.672457      0  
4      54  51.150000      64.289123      16.003063  -13.139123      0

### Merge Weather Data

In [18]:
weather = pd.read_csv('data/external/nyc_weather_2016.csv', index_col=0)

In [19]:
weather = weather[[u'Conditions', u'DateUTC', u'Humidity', u'PrecipitationIn', \
                   u'TemperatureF', u'VisibilityMPH', u'Wind SpeedMPH']]

In [20]:
weather['DateUTC'] = pd.to_datetime(weather['DateUTC'])
weather['hour'] = weather['DateUTC'].dt.hour
weather['month'] = weather['DateUTC'].dt.month
weather['day'] = weather['DateUTC'].dt.day
weather.drop('DateUTC', axis=1, inplace=True)
weather.head()

Conditions  Humidity  PrecipitationIn  TemperatureF  VisibilityMPH  \
0   Overcast      51.0              NaN          42.1           10.0   
1   Overcast      53.0              NaN          41.0           10.0   
2   Overcast      55.0              NaN          41.0           10.0   
3   Overcast      55.0              NaN          41.0           10.0   
4   Overcast      58.0              NaN          39.9           10.0   

  Wind SpeedMPH  hour  month  day  
0           4.6     5      1    1  
1           3.5     6      1    1  
2           4.6     7      1    1  
3           9.2     8      1    1  
4          10.4     9      1    1

In [21]:
weather.drop_duplicates(['hour','month','day'],inplace=True)

In [22]:
len(bus_times)

793681

In [23]:
bus_weather = pd.merge(bus_times, weather, how='left', on=['month','day','hour'])

In [24]:
len(bus_weather)

793681

In [25]:
bus_weather['PrecipitationIn'].fillna(0, inplace=True)
bus_weather.dropna(inplace=True)
bus_weather.sort_values(['bus_line','month','day','hour','minute'], inplace=True)
bus_weather.reset_index(inplace=True, drop=True)
bus_weather.head()

bus_line  direction        date  month  day day_of_week time_period  hour  \
0      B11        1.0  2016-01-01      1    1      Friday       Night     5   
1      B11        2.0  2016-01-01      1    1      Friday      PeakAM     9   
2      B11        2.0  2016-01-01      1    1      Friday      PeakAM     9   
3      B11        1.0  2016-01-01      1    1      Friday      PeakAM     9   
4      B11        2.0  2016-01-01      1    1      Friday      MidDay    10   

   minute  trip_time  avg_trip_time  std_trip_time  delay_time  delay  \
0      27  39.816667      62.360454      20.746053  -22.543787      0   
1      11  39.900000      37.801282       9.328973    2.098718      0   
2      44  38.450000      37.801282       9.328973    0.648718      0   
3      52  53.900000      55.033333       7.915746   -1.133333      0   
4      10  34.700000      58.695643      13.242097  -23.995643      0   

  Conditions  Humidity  PrecipitationIn  TemperatureF  VisibilityMPH  \
0   Overcast      51.0              0.0          42.1           10.0   
1   Overcast      58.0              0.0          39.9           10.0   
2   Overcast      58.0              0.0          39.9           10.0   
3   Overcast      58.0              0.0          39.9           10.0   
4   Overcast      58.0              0.0          39.9           10.0   

  Wind SpeedMPH  
0           4.6  
1          10.4  
2          10.4  
3          10.4  
4           6.9

### Merge Spatial Data

In [26]:
spatial = pd.read_csv('data/external/spatial_features.csv')
spatial.head()

month  totalInjuries route_id  pavementScore  potholeCount
0      1            162      B11         2110.0           237
1     10            110      B11         2110.0           237
2     11            148      B11         2110.0           237
3     12            160      B11         2110.0           237
4      2            114      B11         2110.0           237

In [27]:
len(bus_weather)

788624

In [28]:
bus_weather_spatial = pd.merge(bus_weather, spatial, how='left', left_on=['bus_line','month'], right_on=['route_id','month'])

In [29]:
bus_weather_spatial.drop('route_id', axis=1, inplace=True)
len(bus_weather_spatial)

788624

In [30]:
bus_weather_spatial.head()

bus_line  direction        date  month  day day_of_week time_period  hour  \
0      B11        1.0  2016-01-01      1    1      Friday       Night     5   
1      B11        2.0  2016-01-01      1    1      Friday      PeakAM     9   
2      B11        2.0  2016-01-01      1    1      Friday      PeakAM     9   
3      B11        1.0  2016-01-01      1    1      Friday      PeakAM     9   
4      B11        2.0  2016-01-01      1    1      Friday      MidDay    10   

   minute  trip_time     ...       delay  Conditions  Humidity  \
0      27  39.816667     ...           0    Overcast      51.0   
1      11  39.900000     ...           0    Overcast      58.0   
2      44  38.450000     ...           0    Overcast      58.0   
3      52  53.900000     ...           0    Overcast      58.0   
4      10  34.700000     ...           0    Overcast      58.0   

   PrecipitationIn TemperatureF  VisibilityMPH  Wind SpeedMPH  totalInjuries  \
0              0.0         42.1           10.0            4.6            162   
1              0.0         39.9           10.0           10.4            162   
2              0.0         39.9           10.0           10.4            162   
3              0.0         39.9           10.0           10.4            162   
4              0.0         39.9           10.0            6.9            162   

   pavementScore potholeCount  
0         2110.0          237  
1         2110.0          237  
2         2110.0          237  
3         2110.0          237  
4         2110.0          237  

[5 rows x 23 columns]

### Calculate Previous Trip Time Ratio (2nd prior bus departure)

In [31]:
bus_weather_spatial['prev_trip_ratio'] = (bus_weather_spatial['trip_time']/bus_weather_spatial['avg_trip_time']).shift(2)

In [32]:
bus_weather_spatial.head()

bus_line  direction        date  month  day day_of_week time_period  hour  \
0      B11        1.0  2016-01-01      1    1      Friday       Night     5   
1      B11        2.0  2016-01-01      1    1      Friday      PeakAM     9   
2      B11        2.0  2016-01-01      1    1      Friday      PeakAM     9   
3      B11        1.0  2016-01-01      1    1      Friday      PeakAM     9   
4      B11        2.0  2016-01-01      1    1      Friday      MidDay    10   

   minute  trip_time       ...        Conditions  Humidity  PrecipitationIn  \
0      27  39.816667       ...          Overcast      51.0              0.0   
1      11  39.900000       ...          Overcast      58.0              0.0   
2      44  38.450000       ...          Overcast      58.0              0.0   
3      52  53.900000       ...          Overcast      58.0              0.0   
4      10  34.700000       ...          Overcast      58.0              0.0   

   TemperatureF VisibilityMPH  Wind SpeedMPH  totalInjuries  pavementScore  \
0          42.1          10.0            4.6            162         2110.0   
1          39.9          10.0           10.4            162         2110.0   
2          39.9          10.0           10.4            162         2110.0   
3          39.9          10.0           10.4            162         2110.0   
4          39.9          10.0            6.9            162         2110.0   

   potholeCount prev_trip_ratio  
0           237             NaN  
1           237             NaN  
2           237        0.638492  
3           237        1.055520  
4           237        1.017161  

[5 rows x 24 columns]

In [33]:
bus_weather_spatial.dropna(inplace=True)

In [34]:
bus_weather_spatial.to_csv('data/interim/bus_times_weather_spatial.csv')